# GroupBy and Aggregate Functions

In [1]:
import findspark
findspark.init('/root/Desktop/spark-2.4.3-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('aggs').getOrCreate()

In [4]:
df=spark.read.csv('worldcup.csv',inferSchema=True,header=True)

In [5]:
df.show()

+----+-----+-----+----+------+----+------+--------------+---------+-----------+----------+-----------+----------+
| _c0|Score|Overs| RPO|Target|Inns|Result|    Opposition|   Ground| Start Date|  Match_ID|    Country|Country_ID|
+----+-----+-----+----+------+----+------+--------------+---------+-----------+----------+-----------+----------+
| 412|  250| 48.3|5.15|  null|   1|   won|       v India|  Kolkata| 3 Jan 2013|ODI # 3315|   Pakistan|         7|
| 680|  165| 48.0|3.43|   251|   2|  lost|    v Pakistan|  Kolkata| 3 Jan 2013|ODI # 3315|      India|         6|
| 413|  157| 48.5|3.21|   168|   2|  lost|       v India|    Delhi| 6 Jan 2013|ODI # 3316|   Pakistan|         7|
| 681|  167| 43.4|3.82|  null|   1|   won|    v Pakistan|    Delhi| 6 Jan 2013|ODI # 3316|      India|         6|
| 117|  198| 40.0|4.95|   306|   2|  lost|   v Australia|Melbourne|11 Jan 2013|ODI # 3317|   SriLanka|         8|
|1076|305/5| 50.0|6.10|  null|   1|   won|   v Sri Lanka|Melbourne|11 Jan 2013|ODI # 331

In [6]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Score: string (nullable = true)
 |-- Overs: double (nullable = true)
 |-- RPO: string (nullable = true)
 |-- Target: integer (nullable = true)
 |-- Inns: integer (nullable = true)
 |-- Result: string (nullable = true)
 |-- Opposition: string (nullable = true)
 |-- Ground: string (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Match_ID: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Country_ID: integer (nullable = true)



In [7]:
df.groupBy("Country")

In [9]:
df.groupBy("Country").count().show()

+-----------+-----+
|    Country|count|
+-----------+-----+
|Afghanistan|   90|
|      India|  156|
| Bangladesh|   95|
|SouthAfrica|  135|
| WestIndies|  116|
|  Newzealad|  129|
|    England|  145|
|  Australia|  131|
|   Pakistan|  139|
|   SriLanka|  160|
+-----------+-----+



In [14]:
df.agg({'Score':'sum'}).show()


+----------+
|sum(Score)|
+----------+
|   82956.0|
+----------+



In [15]:
group_data=df.groupby('Country')

In [17]:
group_data.agg({'Score':'count'}).show()

+-----------+------------+
|    Country|count(Score)|
+-----------+------------+
|Afghanistan|          90|
|      India|         156|
| Bangladesh|          95|
|SouthAfrica|         135|
| WestIndies|         116|
|  Newzealad|         129|
|    England|         145|
|  Australia|         131|
|   Pakistan|         139|
|   SriLanka|         160|
+-----------+------------+



In [18]:
from pyspark.sql.functions import countDistinct,avg,stddev

In [19]:
df.select(countDistinct('Score')).show()

+---------------------+
|count(DISTINCT Score)|
+---------------------+
|                  864|
+---------------------+



In [21]:
df.select(avg('Score').alias('Average Score')).show()

+------------------+
|     Average Score|
+------------------+
|208.95717884130983|
+------------------+



In [31]:
df.select(stddev('Score')).show()

+------------------+
|stddev_samp(Score)|
+------------------+
| 55.60621540818044|
+------------------+



In [24]:
from pyspark.sql.functions import format_number

In [29]:
score_std=df.select(stddev('Score').alias('std'))

In [32]:
score_std.select(format_number('std',2).alias('std')).show()

+-----+
|  std|
+-----+
|55.61|
+-----+



In [35]:
df.orderBy('Country').show()

+----+-----+-----+----+------+----+------+------------+------------+-----------+----------+-----------+----------+
| _c0|Score|Overs| RPO|Target|Inns|Result|  Opposition|      Ground| Start Date|  Match_ID|    Country|Country_ID|
+----+-----+-----+----+------+----+------+------------+------------+-----------+----------+-----------+----------+
|1216|302/5| 50.0|6.04|  null|   1|   won|    v U.A.E.|Kuala Lumpur| 2 May 2014|ODI # 3488|Afghanistan|        40|
|1236|271/6| 50.0|5.42|  null|   1|   won|  v Zimbabwe|    Bulawayo|18 Oct 2015|ODI # 3694|Afghanistan|        40|
|1217|223/9| 50.0|4.46|  null|   1|  lost|  v Zimbabwe|    Bulawayo|18 Jul 2014|ODI # 3503|Afghanistan|        40|
|1207|203/3| 33.5|6.00|   200|   2|   won|  v Scotland|     Sharjah| 6 Mar 2013|ODI # 3341|Afghanistan|        40|
|1218|256/7| 50.0|5.12|  null|   1|  lost|  v Zimbabwe|    Bulawayo|20 Jul 2014|ODI # 3504|Afghanistan|        40|
|1209| 95/2| 17.5|5.32|    90|   2|   won|     v Kenya|     Sharjah| 2 Oct 2013|

In [36]:
df.orderBy(df['Country'].desc()).show()

+---+-----+-----+----+------+----+------+--------------+-------------+-----------+----------+----------+----------+
|_c0|Score|Overs| RPO|Target|Inns|Result|    Opposition|       Ground| Start Date|  Match_ID|   Country|Country_ID|
+---+-----+-----+----+------+----+------+--------------+-------------+-----------+----------+----------+----------+
| 20|242/7| 50.0|4.84|  null|   1|  lost|    v Pakistan|   Gros Islet|24 Jul 2013|ODI # 3396|WestIndies|         4|
| 40|  122| 33.4|3.62|  null|   1|  lost|v South Africa|  East London|21 Jan 2015|ODI # 3587|WestIndies|         4|
| 21|  211| 48.5|4.32|  null|   1|  lost|       v India|        Kochi|21 Nov 2013|ODI # 3436|WestIndies|         4|
|  5|  257| 49.5|5.15|   275|   2|  lost|   v Australia|    Melbourne|10 Feb 2013|ODI # 3334|WestIndies|         4|
| 22|289/8| 49.3|5.83|   289|   2|   won|       v India|Visakhapatnam|24 Nov 2013|ODI # 3437|WestIndies|         4|
|  6|337/4| 50.0|6.74|  null|   1|   won|    v Zimbabwe|  St George's|22